In [ ]:
import tensorflow as tf
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

In [ ]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

In [ ]:
import sys
sys.path.append('/kaggle/input/images10/cifar-10-batches-py')

In [ ]:

# Path to the extracted CIFAR-10 dataset folder in Kaggle
dataset_folder = '/kaggle/input/images10/cifar-10-batches-py'

# Function to load a batch of CIFAR-10 dataset
def load_cifar_batch(filename):
    with open(filename, 'rb') as f:
        data_dict = pickle.load(f, encoding='bytes')
        images = data_dict[b'data']
        labels = data_dict[b'labels']
        # Reshape the images
        images = images.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
        labels = np.array(labels)
        return images, labels

# Load all training batches (1 to 5)
x_train, y_train = [], []
for i in range(1, 6):
    batch_file = os.path.join(dataset_folder, f'data_batch_{i}')
    images, labels = load_cifar_batch(batch_file)
    x_train.append(images)
    y_train.append(labels)

# Concatenate all batches to form the complete training set
x_train = np.concatenate(x_train)
y_train = np.concatenate(y_train)

# Load the test batch
x_test, y_test = load_cifar_batch(os.path.join(dataset_folder, 'test_batch'))

# Check the shapes of the loaded data
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# flatten the label values
y_train, y_test = y_train.flatten(), y_test.flatten()

In [ ]:
# visualize data by plotting images
fig, ax = plt.subplots(5, 5)
k = 0

for i in range(5):
	for j in range(5):
		ax[i][j].imshow(x_train[k], aspect='auto')
		k += 1

plt.show()

In [ ]:
# number of classes
K = len(set(y_train))

# calculate total number of classes
# for output layer
print("number of classes:", K)

# Build the model using the functional API
# input layer
i = Input(shape=x_train[0].shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dropout(0.2)(x)

# Hidden layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)

# last hidden layer i.e.. output layer
x = Dense(K, activation='softmax')(x)

model = Model(i, x)

# model description
model.summary()

In [ ]:
# Compile
model.compile(optimizer='adam',
			loss='sparse_categorical_crossentropy',
			metrics=['accuracy'])

In [ ]:
# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Fit
r = model.fit(
x_train, y_train, validation_data=(x_test, y_test), epochs=50, 
    callbacks=[early_stopping]  # Add early stopping here
)

In [ ]:
# Fit with data augmentation
# the previous model.fit()
# it will CONTINUE training where it left off
batch_size = 32
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

train_generator = data_generator.flow(x_train, y_train, batch_size)
steps_per_epoch = x_train.shape[0] // batch_size

# Fit with data augmentation (continuing training)
r = model.fit(
    train_generator,
    validation_data=(x_test, y_test),
    steps_per_epoch=steps_per_epoch,
    epochs=50
)

In [ ]:
# Plot accuracy per iteration
plt.plot(r.history['accuracy'], label='acc', color='red')
plt.plot(r.history['val_accuracy'], label='val_acc', color='green')
plt.legend()

In [ ]:
# label mapping
labels = '''airplane automobile bird cat deer dog frog horse ship truck'''.split()

# Select the image number for the frog
image_number = 2  # Change this to the index of your frog image

# Display the image
plt.imshow(x_test[image_number])
plt.axis('off')  # Turn off axis for better visualization
plt.show()

# Load the image in an array
n = np.array(x_test[image_number])

# Reshape it for the model
p = n.reshape(1, 32, 32, 3)

# Make a prediction
predicted_probabilities = model.predict(p)
predicted_label = labels[predicted_probabilities.argmax()]

# Load the original label
original_label = labels[y_test[image_number]]

# Display the results
print(f"Original label is '{original_label}' and predicted label is '{predicted_label}'")
print("Output probabilities:", predicted_probabilities)